In [1]:
import pandas as pd

In [ ]:
data=pd.read_csv("/content/Doceree-HCP_Train.csv", encoding='latin1')
data.tail()

In [66]:
data.head()

,ID,DEVICETYPE,PLATFORM_ID,BIDREQUESTIP,USERPLATFORMUID,USERCITY,USERZIPCODE,USERAGENT,PLATFORMTYPE,CHANNELTYPE,URL,KEYWORDS,TAXONOMY,IS_HCP
0,1001,Desktop,2,170.173.0.22,6974dcaa-f932-480e-9fb5-c52e20e1393a,Portland,97206.0,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,Online Medical Journal,Website,https://www.cancertherapyadvisor.com/home/canc...,General|Endocrine|False|Medicine|Surgery|Urolo...,NaN,0.0
1,1002,Desktop,2,65.216.253.25,c12f3f8f-8fcf-484a-90e1-1ac04db8cdcf,Arlington,22202.0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,Online Medical Journal,Website,https://www.cancertherapyadvisor.com/home/deci...,Bone Marrow|Radiography|Chronic|Oncology|Psych...,NaN,0.0
2,1003,Desktop,2,66.232.79.22,a698de4b-e200-46dd-b5fb-40402175ae18,New Meadows,83654.0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,Online Medical Journal,Website,https://www.cancertherapyadvisor.com/home/canc...,General|Endocrine|False|Medicine|Surgery|Urolo...,NaN,0.0
3,1004,Desktop,3,137.54.125.246,45967533-75c8-4fbd-a00c-e6ff20447aaa,NaN,229114624.0,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,Online Medical Journal,Website,https://globalrph.com/medcalcs/warfarin-mainte...,Dental|Total|Clinical|Pharmacology|Physicians|...,2084P0800X,1.0
4,1005,Mobile,7,174.202.231.99,a17e25be-532d-4cf5-b916-9308c8c3961f,Houston,77008.0,Mozilla/5.0 (iPhone; CPU iPhone OS 16_1_1 like...,Online Medical Journal,Website,https://www.cureus.com/articles/58184-a-review...,Critical Care|Emergency Medicine|General Pract...,NaN,0.0


In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report,confusion_matrix
import joblib
import numpy as np

In [4]:
train_data = data.drop(['USERPLATFORMUID'], axis=1)

In [10]:
train_data=train_data.dropna(subset=['IS_HCP'])
train_data.isnull().sum()

ID                  0
DEVICETYPE          0
PLATFORM_ID         0
BIDREQUESTIP        0
USERCITY         6359
USERZIPCODE      4592
USERAGENT           2
PLATFORMTYPE        0
CHANNELTYPE         0
URL                 0
KEYWORDS            0
TAXONOMY        81623
IS_HCP              0
dtype: int64

In [11]:
train_data['IS_HCP'].isnull().value_counts()

False    113936
Name: IS_HCP, dtype: int64

In [13]:
# Handling missing values
train_data.fillna('', inplace=True)

In [14]:
# Label encoding categorical variables
devicetype_le = LabelEncoder()
train_data['DEVICETYPE'] = devicetype_le.fit_transform(train_data['DEVICETYPE']).astype(str)

In [15]:
# Add new value 'Desktop' to classes
le_classes = devicetype_le.classes_.tolist()
le_classes.append('Desktop')
devicetype_le.classes_ = np.asarray(le_classes)

In [16]:
# Label encoding categorical variables
usercity_le = LabelEncoder()
train_data['USERCITY'] = usercity_le.fit_transform(train_data['USERCITY'])

In [17]:
# Add new value 'Peterstown' to classes
le_classes = usercity_le.classes_.tolist()
le_classes.append('Peterstown')
le_classes.append('Boaz')
le_classes.append('Kinder')
le_classes.append('Old Westbury')
le_classes.append('North Conway')
le_classes.append('Byram')
le_classes.append('Martinez')
le_classes.append('Keedysville')
le_classes.append('Linden')
le_classes.append('Clarks Hill')
le_classes.append('Elburn')
le_classes.append('Bayville')
le_classes.append('Blue River')
le_classes.append('Dover-Foxcroft')
le_classes.append('Efland')
usercity_le.classes_ = np.asarray(le_classes)

In [18]:
userzipcode_le = LabelEncoder()
train_data['USERZIPCODE'] = userzipcode_le.fit_transform(train_data['USERZIPCODE'].astype(str))

In [19]:
useragent_le = LabelEncoder()
train_data['USERAGENT'] = useragent_le.fit_transform(train_data['USERAGENT'])

platformtype_le = LabelEncoder()
train_data['PLATFORMTYPE'] = platformtype_le.fit_transform(train_data['PLATFORMTYPE'])

In [20]:
channeltype_le = LabelEncoder()
train_data['CHANNELTYPE'] = channeltype_le.fit_transform(train_data['CHANNELTYPE'])

url_le = LabelEncoder()
train_data['URL'] = url_le.fit_transform(train_data['URL'])

keywords_le = LabelEncoder()
train_data['KEYWORDS'] = keywords_le.fit_transform(train_data['KEYWORDS'])

In [21]:
# Create an instance of LabelEncoder
is_hcp_le = LabelEncoder()

In [22]:
# Splitting into features and target variable
X = train_data.drop(['IS_HCP', 'TAXONOMY', 'BIDREQUESTIP'], axis=1)
X = X.drop(['URL', 'USERAGENT'], axis=1)
y = is_hcp_le.fit_transform(train_data['IS_HCP'])

In [25]:
from collections import Counter

value_counts = Counter(y)

# Print the value counts
for value, count in value_counts.items():
    print(f"{value}: {count}")

0: 79756
1: 34180


In [26]:
# Step 4: Feature Engineering
# Count the number of keywords in the URL
X['NUM_KEYWORDS'] = train_data['URL'].astype(str).apply(lambda x: len(x.split(',')))

In [27]:
X

,ID,DEVICETYPE,PLATFORM_ID,USERCITY,USERZIPCODE,PLATFORMTYPE,CHANNELTYPE,KEYWORDS,NUM_KEYWORDS
0,1001,0,2,3178,10914,3,0,458,1
1,1002,0,2,131,1901,3,0,108,1
2,1003,0,2,2702,9157,3,0,458,1
3,1004,0,3,0,1960,3,0,273,1
4,1005,1,7,1768,8360,3,0,257,1
...,...,...,...,...,...,...,...,...,...
113932,114933,0,2,3068,1333,3,0,148,1
113933,114934,0,2,4044,9979,3,0,67,1
113934,114935,0,7,0,11062,3,0,921,1
113935,114936,1,2,4323,1468,3,0,1679,1


In [28]:
# Extract browser information from USERAGENT
X['BROWSER'] = train_data['USERAGENT'].astype(str).apply(lambda x: x.split()[0])

In [29]:
# Count the number of characters in USERCITY
X['CITY_LENGTH'] = X['USERCITY'].astype(str).apply(lambda x: len(x))


In [62]:
X

,ID,DEVICETYPE,PLATFORM_ID,USERCITY,USERZIPCODE,PLATFORMTYPE,CHANNELTYPE,KEYWORDS,NUM_KEYWORDS,BROWSER,CITY_LENGTH
0,1001,0,2,3178,10914,3,0,458,1,2858,4
1,1002,0,2,131,1901,3,0,108,1,2673,3
2,1003,0,2,2702,9157,3,0,458,1,2673,4
3,1004,0,3,0,1960,3,0,273,1,2946,1
4,1005,1,7,1768,8360,3,0,257,1,4050,4
...,...,...,...,...,...,...,...,...,...,...,...
113932,114933,0,2,3068,1333,3,0,148,1,2589,4
113933,114934,0,2,4044,9979,3,0,67,1,3019,4
113934,114935,0,7,0,11062,3,0,921,1,2651,1
113935,114936,1,2,4323,1468,3,0,1679,1,3848,4


In [30]:
# Step 5: Model Selection
model = RandomForestClassifier()

In [31]:
# Step 6: Model Training and Evaluation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier()

In [32]:
# Save the trained model
joblib.dump(model, 'model.pkl')

['model.pkl']

In [33]:
# Load the saved model
model = joblib.load('model.pkl')

In [34]:
predictions = model.predict(X_val)

In [35]:
# Evaluating the model
print("Accuracy:", accuracy_score(y_val, predictions))
print("Classification Report:")
print(classification_report(y_val, predictions))
print(confusion_matrix(y_val, predictions))

Accuracy: 0.9822713708969634
Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.99      0.99     15906
           1       0.98      0.96      0.97      6882

    accuracy                           0.98     22788
   macro avg       0.98      0.98      0.98     22788
weighted avg       0.98      0.98      0.98     22788

[[15762   144]
 [  260  6622]]


In [96]:
model2=RandomForestClassifier(max_depth=6, n_estimators=400)

In [97]:
# Step 6: Model Training and Evaluation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)
model2.fit(X_train, y_train)

RandomForestClassifier(max_depth=6, n_estimators=400)

In [98]:
predictions = model2.predict(X_val)
# Evaluating the model
print("Accuracy:", accuracy_score(y_val, predictions))
print("Classification Report:")
print(classification_report(y_val, predictions))
print(confusion_matrix(y_val, predictions))

Accuracy: 0.8309294637371639
Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.98      0.89     23915
           1       0.90      0.49      0.64     10266

    accuracy                           0.83     34181
   macro avg       0.86      0.73      0.76     34181
weighted avg       0.84      0.83      0.81     34181

[[23373   542]
 [ 5237  5029]]


In [82]:
# Step 5: Prediction and Evaluation on Test Data
test_data = pd.read_csv("/content/Doceree-HCP_Test.csv")

In [83]:
test_data

,ID,DEVICETYPE,PLATFORM_ID,BIDREQUESTIP,USERPLATFORMUID,USERCITY,USERZIPCODE,USERAGENT,PLATFORMTYPE,CHANNELTYPE,URL,KEYWORDS
0,115501,Desktop,2,75.189.231.103,0d5041ff-f0b6-4d1a-9ad7-0a29f7d485b4,Fayetteville,28305.0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6...,Online Medical Journal,Website,https://www.clinicaladvisor.com/home/features/...,Family Practice|Drainage|Clinical|Dermatology|...
1,115502,Mobile,2,24.101.33.158,c8396dd0-969f-4d99-a40b-b7bb1f516154,Conneaut Lake,16316.0,Mozilla/5.0 (iPhone; CPU iPhone OS 15_6_1 like...,Online Medical Journal,Website,https://www.ophthalmologyadvisor.com/topics/ca...,General|Clinical|Operative|Medicine|Cardiology...
2,115503,Desktop,2,172.118.216.142,3c97a081-6518-43f8-9f26-369759cfb471,Covina,91724.0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,Online Medical Journal,Website,https://www.psychiatryadvisor.com/author/tori-...,Abortion|Anxiety Disorders|Apnea|False|Trauma|...
3,115504,Desktop,7,71.105.120.171,3e2578c8-f794-41af-a38c-c5cfb3c0f014,Brooklyn,11226.0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,Online Medical Journal,Website,https://www.cureus.com/articles/105482-diverti...,Health|Male|Neurological Surgery|Otolaryngolog...
4,115505,Desktop,2,73.82.211.73,ec2ae7ce-6a8c-4156-98a7-07203e60f483,Marietta,30062.0,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,Online Medical Journal,Website,https://www.renalandurologynews.com/home/confe...,chronic kidney disease|pain|nephrology|disease...
...,...,...,...,...,...,...,...,...,...,...,...,...
28488,143989,Desktop,2,69.202.233.241,78ce4bbe-3885-4c14-b945-a0ea1e4574f4,Brooklyn,11215.0,Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebK...,Online Medical Journal,Website,https://www.renalandurologynews.com/home/news/...,Transplantation|Psychiatry|Heart Failure|Angio...
28489,143990,Desktop,7,75.4.190.65,a53799e1-b279-40cf-b467-11b4d659e325,Miami,33178.0,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,Online Medical Journal,Website,https://www.cureus.com/publish/articles/144818...,small|hospitals|nephrology|emergency medicine|...
28490,143991,Desktop,7,137.52.180.45,acc732a8-2ee9-4e2c-9933-70e34db48101,Fort Lauderdale,33314.0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,Online Medical Journal,Website,https://www.cureus.com/registrations/continue?...,small|hospitals|nephrology|emergency medicine|...
28491,143992,Desktop,8,66.249.66.4,dce21294-b105-4abb-b145-4e62d71def44,NaN,NaN,Mozilla/5.0 (Linux; Android 6.0.1; Nexus 5X Bu...,Online Medical Journal,Website,https://radrounds.com/radiology-case-images-te...,the|disease|pain|radiology|lung abscess|dyspar...


In [84]:
test_data = test_data.drop([ 'USERPLATFORMUID'], axis=1)
test_data.fillna('', inplace=True)

In [85]:
devicetype_le=LabelEncoder()
usercity_le=LabelEncoder()
userzipcode_le=LabelEncoder()
useragent_le=LabelEncoder()
platformtype_le=LabelEncoder()
channeltype_le=LabelEncoder()
url_le=LabelEncoder()
keywords_le=LabelEncoder()


In [86]:
# Label encoding categorical variables
test_data['DEVICETYPE'] = devicetype_le.fit_transform(test_data['DEVICETYPE']).astype(str)
test_data['USERCITY'] = usercity_le.fit_transform(test_data['USERCITY'])
test_data['USERZIPCODE'] = userzipcode_le.fit_transform(test_data['USERZIPCODE'].astype(str))
test_data['USERAGENT'] = useragent_le.fit_transform(test_data['USERAGENT'])
test_data['PLATFORMTYPE'] = platformtype_le.fit_transform(test_data['PLATFORMTYPE'])
test_data['CHANNELTYPE'] = channeltype_le.fit_transform(test_data['CHANNELTYPE'])
test_data['URL'] = url_le.fit_transform(test_data['URL'])
test_data['KEYWORDS'] = keywords_le.fit_transform(test_data['KEYWORDS'])

In [87]:
# Feature engineering on the test set
test_data['NUM_KEYWORDS'] = test_data['URL'].astype(str).apply(lambda x: len(x.split(',')))
test_data['BROWSER'] = test_data['USERAGENT'].astype(str).apply(lambda x: x.split()[0])
test_data['CITY_LENGTH'] = test_data['USERCITY'].astype(str).apply(lambda x: len(x))

In [88]:
test_data=test_data.drop(['URL', 'USERAGENT','BIDREQUESTIP'], axis=1)

In [48]:
test_data

,ID,DEVICETYPE,PLATFORM_ID,USERCITY,USERZIPCODE,PLATFORMTYPE,CHANNELTYPE,KEYWORDS,NUM_KEYWORDS,BROWSER,CITY_LENGTH
0,115501,0,2,858,1621,3,0,326,1,1231,3
1,115502,1,2,560,614,3,0,354,1,2120,3
2,115503,0,2,593,6202,3,0,0,1,1329,3
3,115504,0,7,324,196,3,0,412,1,1357,3
4,115505,0,2,1610,1848,3,0,877,1,1444,4
...,...,...,...,...,...,...,...,...,...,...,...
28488,143989,0,2,324,184,3,0,818,1,1395,3
28489,143990,0,7,1694,2245,3,0,1490,1,1492,4
28490,143991,0,7,904,2262,3,0,1494,1,1329,3
28491,143992,0,8,0,0,3,0,1639,1,1035,1


In [92]:
y_test_pred=model.predict(test_data)

In [93]:
y_test_pred

array([1, 0, 0, ..., 0, 0, 0])

In [94]:
# Prepare submission files
df = pd.DataFrame({"ID":test_data["ID"],'IS_HCP': y_test_pred})
#scoring_solution_df.to_csv('Scoring_Solution.csv', index=False)

df


,ID,IS_HCP
0,115501,1
1,115502,0
2,115503,0
3,115504,0
4,115505,1
...,...,...
28488,143989,0
28489,143990,0
28490,143991,0
28491,143992,0


In [95]:
result=df[df["IS_HCP"]==1]
result

,ID,IS_HCP
0,115501,1
4,115505,1
5,115506,1
7,115508,1
8,115509,1
...,...,...
28481,143982,1
28483,143984,1
28484,143985,1
28485,143986,1


In [75]:
result.to_csv('hcp_ids.csv', index=False)

In [67]:
model.predict([[1004,0,3,0,1960,3,0,273,1,2946,1]])

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


array([1])